In [24]:
from pyspark import (SparkConf, SparkContext)
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from tqdm import tqdm

import pyspark.sql.functions as F
import pyspark
import random
import pickle
import pandas as pd
import time

In [2]:
ss = SparkSession.builder.appName('testa').getOrCreate()

In [26]:
for i in tqdm(range(0, 79)): 
    with open('./pandaset_data/' + str(i) + '.pkl', 'rb') as p_f:
            data = pickle.load(p_f)

    refined_data = data.drop(['uuid', 'attributes.pedestrian_behavior', 'attributes.rider_status', 
                              'attributes.object_motion', 'attributes.pedestrian_age', 'cuboids.sibling_id', 
                              'cuboids.sensor_id', 'camera_used', 'stationary'], axis=1)
    
    if i == 0:
        df = pd.DataFrame(refined_data)
    else:
        df = pd.concat([df, pd.DataFrame(refined_data)], ignore_index=True)
    
print(df.info())
df.to_csv("pandaspark.csv")

100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 431.71it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11751 entries, 0 to 11750
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         11751 non-null  object 
 1   yaw           11751 non-null  float64
 2   position.x    11751 non-null  float64
 3   position.y    11751 non-null  float64
 4   position.z    11751 non-null  float64
 5   dimensions.x  11751 non-null  float64
 6   dimensions.y  11751 non-null  float64
 7   dimensions.z  11751 non-null  float64
dtypes: float64(7), object(1)
memory usage: 734.6+ KB
None


In [45]:
schema = StructType([
    StructField('label', StringType(), True),
    StructField('yaw', FloatType(), True),
    StructField('position.x', FloatType(), True),
    StructField('position.y', FloatType(), True),
    StructField('position.z', FloatType(), True),
    StructField('dimensions.x', FloatType(), True),
    StructField('dimensions.y', FloatType(), True),
    StructField('dimensions.z', FloatType(), True)
])

ddf = ss.read.schema(schema = schema).csv("pandaspark.csv", header=True)
ddf.printSchema()

# ddf = ddf.where((ddf.label == "Car") | (ddf.label == "Pedestrian")).drop("yaw", "dimensions.z").show(truncate=False)
ddf = ddf.where((ddf.label == "Car") | (ddf.label == "Pedestrian")).drop("yaw", "dimensions.z").toPandas().to_csv('integrated_panda_car_ped.csv')

root
 |-- label: string (nullable = true)
 |-- yaw: float (nullable = true)
 |-- position.x: float (nullable = true)
 |-- position.y: float (nullable = true)
 |-- position.z: float (nullable = true)
 |-- dimensions.x: float (nullable = true)
 |-- dimensions.y: float (nullable = true)
 |-- dimensions.z: float (nullable = true)

